In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time
from logger import setup_logger, log_info, log_debug, log_error

In [2]:
load_dotenv()
setup_logger()

In [3]:
key = os.getenv("OPENAI_API_KEY")

In [4]:
my_client = OpenAI(api_key=key)

In [5]:
def time_this_function(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"Function `{func.__name__}` took {end - start:.3f} seconds to run.")
        return res

    return wrapper

In [6]:
@time_this_function
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]

    response = my_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    log_debug(response)

    return response.choices[0].message.content


@time_this_function
def stream_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]

    response = my_client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
        temperature=0,
    )

    log_debug(response)

    for resp_chunk in response:
        curr_chunk = resp_chunk.choices[0].delta.content
        if curr_chunk is not None:
            yield curr_chunk


@time_this_function
def converse(
    prompt,
    messages=None,
    model="gpt-3.5-turbo",
    max_tokens=3000,
    temperature=0,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    if messages is None:
        messages = list()

    messages.append({"role": "user", "content": prompt})

    response = (
        my_client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty,
        )
        .choices[0]
        .message.content
    )

    messages.append({"role": "assistant", "content": response})

    return response, messages

In [7]:
@time_this_function
def create_assistant(assistant_name: str, instruction: str, model: str = "gpt-3.5-turbo"):
    my_assistant = my_client.beta.assistants.create(model=model, instructions=instruction, name=assistant_name)
    log_info(my_assistant)

    return my_assistant.id


@time_this_function
def get_assistant_from_id(assistant_id: str):
    my_assistant = my_client.beta.assistants.retrieve(assistant_id)
    log_info(my_assistant)

    return my_assistant.id


@time_this_function
def delete_assistant_with_id(assistant_id: str):
    response = my_client.beta.assistants.delete(assistant_id)
    log_info(response)

    return response.deleted


@time_this_function
def create_assistant_thread(user_msg: str):
    thread = my_client.beta.threads.create(messages=[{"role": "user", "content": user_msg}])
    log_info(thread)

    return thread.id


@time_this_function
def create_run_from_thread(thread_id: str, assistant_id: str):
    run = my_client.beta.threads.runs.create(thread_id=thread_id, assistant_id=assistant_id)
    log_info(run)

    return run.id


@time_this_function
def poll_run_till_complete(thread_id: str, run_id: str):
    while True:
        run = my_client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

        if run.status not in ["queued", "in_progress"]:
            log_info(run)
            return run

        time.sleep(1)


@time_this_function
def list_messages_in_thread(thread_id: str):
    messages = my_client.beta.threads.messages.list(thread_id=thread_id)
    log_info(messages)

    return messages

In [8]:
ONE_GAME_EVENTS = "It is a football match in the 2012 season of the first division league of germany. The match is played in the home turf of Borussia Dortmund on 2011-08-05, between home team Borussia Dortmund and away team Hamburg SV. On minute 2, the game is being played on Left side of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was mladen petric who used his left foot for the shot with the secondary player involved being gokhan tore.  The shot was off target and was flying High and wide. On minute 14, the game is being played on Outside the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was shinji kagawa who used his right foot for the shot with the secondary player involved being mario gotze.  The shot was off target and was flying Top right corner. On minute 17, the game is being played on Left side of the box in the field, and Attempt happens accompanied with Key Pass. This resulted in a goal for Borussia Dortmund, which was accompanied by an assist via Pass. The primary player involved in the event was kevin grosskreutz who used his left foot for the shot with the secondary player involved being mario gotze.  The shot was on target and was placed at the Bottom right corner of the goal. On minute 19, the game is being played on Outside the box in the field, and Attempt happens. The primary player involved in the event was mats hummels who used his right foot for the shot. The shot was blocked by the opponent team. On minute 25, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was shinji kagawa who used his right foot for the shot with the secondary player involved being lukasz piszczek.  The shot had hit the bar. On minute 26, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was mats hummels who used his head for the shot with the secondary player involved being chris lowe.  The shot was on target and was placed at the Centre of the goal of the goal. On minute 28, the game is being played on Left side of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was shinji kagawa who used his right foot for the shot with the secondary player involved being kevin grosskreutz.  The shot was off target and was flying Misses to the right. On minute 29, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. This resulted in a goal for Borussia Dortmund, which was accompanied by an assist via Pass. The primary player involved in the event was mario gotze who used his right foot for the shot with the secondary player involved being robert lewandowski.  The shot was on target and was placed at the Centre of the goal of the goal. On minute 32, the game is being played on nan in the field, and Corner happens. The primary player involved in the event was ilkay gundogan with the secondary player involved being ilkay gundogan.  On minute 39, the game is being played on Left side of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was robert lewandowski who used his head for the shot with the secondary player involved being lukasz piszczek.  The shot was off target and was flying Misses to the right. On minute 42, the game is being played on nan in the field, and Substitution happens. The player jose paolo guerrero is substituted by anis benhatira.  On minute 45, the game is being played on Right side of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was ilkay gundogan who used his right foot for the shot with the secondary player involved being mario gotze.  The shot was off target and was flying Misses to the left. On minute 48, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was kevin grosskreutz who used his right foot for the shot with the secondary player involved being mario gotze.  The shot was on target and was placed at the Top centre of the goal of the goal. On minute 49, the game is being played on Very close range in the field, and Attempt happens. This resulted in a goal for Borussia Dortmund, which was accompanied by an assist via None. The primary player involved in the event was kevin grosskreutz who used his right foot for the shot. The shot was on target and was placed at the Bottom right corner of the goal. On minute 54, the game is being played on Outside the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was shinji kagawa who used his right foot for the shot with the secondary player involved being kevin grosskreutz.  The shot was blocked by the opponent team. On minute 57, the game is being played on nan in the field, and Corner happens. The primary player involved in the event was felipe santana with the secondary player involved being felipe santana.  On minute 60, the game is being played on Defensive half in the field, and Free kick happens. The primary player involved in the event was anis benhatira. On minute 64, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was michael mancienne who used his right foot for the shot with the secondary player involved being gokhan tore.  The shot was blocked by the opponent team. On minute 74, the game is being played on nan in the field, and Corner happens. The primary player involved in the event was chris lowe with the secondary player involved being chris lowe.  On minute 76, the game is being played on nan in the field, and Substitution happens. The player chris lowe is substituted by ivan perisic.  On minute 79, the game is being played on Centre of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was marcell jansen who used his head for the shot with the secondary player involved being anis benhatira.  The shot was blocked by the opponent team. On minute 80, the game is being played on Centre of the box in the field, and Attempt happens. This resulted in a goal for Hamburg SV, which was accompanied by an assist via None. The primary player involved in the event was robert tesche who used his left foot for the shot. The shot was on target and was placed at the Bottom left corner of the goal. On minute 80, the game is being played on Left side of the box in the field, and Attempt happens accompanied with Key Pass. The primary player involved in the event was marcell jansen who used his left foot for the shot with the secondary player involved being gokhan tore.  The shot was on target and was placed at the Centre of the goal of the goal. On minute 82, the game is being played on Outside the box in the field, and Attempt happens. The primary player involved in the event was ivan perisic who used his right foot for the shot. The shot was off target and was flying Misses to the right. On minute 87, the game is being played on More than 35 yards in the field, and Attempt happens. The primary player involved in the event was kevin grosskreutz who used his right foot for the shot. The shot was off target and was flying Too high. On minute 90, the game is being played on nan in the field, and Substitution happens. The player shinji kagawa is substituted by sebastian kehl.  The final result of the game is 1 goals for Hamburg SV vs 3 goals for Borussia Dortmund. The winner is the home team Borussia Dortmund."

In [ ]:
ASSISTANT_INSTRUCTION = (
    "You are a football commentator bot in the 5 major leagues in Europe, who summarizes the major events "
    "of a game. You are given the contexts for the game, as well as the final results of the game. Use the context and search the "
    "latest data available to set up the environment, then summarize the game, and finally explain the result. While "
    "summarizing the game, you should mention the major events that had impacted the game. If you find the stadium names, "
    "or the shirt number of the players, you should also include those in your summary. Try to keep the commentary "
    "interesting and engaging for the audience while maintaining a 5 minute spoken length for the commentary. "
)

In [9]:
@time_this_function
def create_run_delete_assistant():
    assistant_name = "Soccer Commentator Bot"
    assistant_id = create_assistant(assistant_name, ASSISTANT_INSTRUCTION)
    retrieved_assistant_id = get_assistant_from_id(assistant_id)
    assert retrieved_assistant_id == assistant_id
    thread_id = create_assistant_thread(ONE_GAME_EVENTS)
    run_id = create_run_from_thread(thread_id, assistant_id)
    run_details = poll_run_till_complete(thread_id, run_id)
    if delete_assistant_with_id(assistant_id):
        log_info(f"Deleted assistant with id: {assistant_id}")
    print(run_details)
    msgs = list_messages_in_thread(thread_id)
    print(msgs)

In [10]:
@time_this_function
def assistant_test_run():
    assistant_name = "Soccer Commentator Bot"
    assistant_instruction = ASSISTANT_INSTRUCTION
    assistant_id = create_assistant(assistant_name, assistant_instruction)
    # assistant_id = "asst_8rrQUDUGROgPfMrSZxo3Fb8G"
    retrieved_assistant_id = get_assistant_from_id(assistant_id)
    assert retrieved_assistant_id == assistant_id
    # print(f"{ONE_GAME_EVENTS=}")
    response = create_run_delete_assistant(assistant_id, ONE_GAME_EVENTS)
    print(response)
    # delete_assistant_with_id(assistant_id)

In [11]:
@time_this_function
def chat_test_run():
    prompt = f"How are you feeling today?"
    response = get_completion(prompt)
    print(response)

In [12]:
# chat_test_run()

In [13]:
create_run_delete_assistant()

Function `create_assistant` took 0.475 seconds to run.
Function `get_assistant_from_id` took 0.205 seconds to run.
Function `create_assistant_thread` took 0.320 seconds to run.
Function `create_run_from_thread` took 0.435 seconds to run.
Function `poll_run_till_complete` took 4.797 seconds to run.
Run(id='run_pxT2pmvAPlMYDVlhgvO4LPu2', assistant_id='asst_33zMYiOWBXm0STTUrY70ssHh', cancelled_at=None, completed_at=1713383993, created_at=1713383988, expires_at=None, failed_at=None, file_ids=[], instructions='You are a football commentator bot in the 5 major leagues in Europe, who summarizes the major events of a game. You are given the contexts for the game, as well as the final results of the game. Use the context and search the latest data available to set up the environment, then summarize the game, and finally explain the result.', last_error=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, started_at=1713383988, status='completed', thread_id='threa